In [30]:
import os
from datasets import load_dataset
import datasets
from datasets import Value, Sequence
import sys
import math
import transformers

cache_dir = '/users/PDS0352/wyang107/project/LCEG/model_cache/data'

In [3]:
# InfiniteBench = load_dataset('xinrongzhang2022/InfiniteBench', cache_dir=cache_dir, streaming= True)
ft = datasets.Features({"id": Value("int64"), "context": Value("string"), "input": Value("string"), "answer": Sequence(Value("string")), "options": Sequence(Value("string"))})
InfiniteBench = load_dataset("xinrongzhang2022/InfiniteBench", features=ft, cache_dir=cache_dir, streaming=True)

In [32]:
tokenizer = transformers.AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [37]:
# 'gsm100'
# 'codeU'
# 'multidoc_qa'  'legal_contract_qa', 'financial_qa', 'natural_question' 'scientific_qa' 
# 'news_summ', 'patent_summ' 'tv_show_summ' 'review_summ',  'meeting_summ' 
dataset = load_dataset('L4NLP/LEval', name='gsm100'  ,split='test', cache_dir=cache_dir, streaming= True)

In [38]:
for data in dataset:
    print(data.keys())
    print(len(data['instructions']), len(data['outputs']), data['evaluation'])
    tokens = tokenizer.encode(data['input'])
    print(len(tokens))
    break

dict_keys(['instructions', 'input', 'outputs', 'source', 'evaluation'])
1 1 exam
5451


In [39]:
def get_dataset(path='LongBench', name='narrativeqa'):
    new_dataset = list()
    if path == 'LongBench':
        dataset = load_dataset('THUDM/LongBench',name=name, split='test', cache_dir=cache_dir, streaming= True)
        for obj in dataset:
            new_dataset.append({"input": obj['input'], "length": len(tokenizer.encode(obj['context'])), 
                            'context':obj['context'] , 'answers':obj['answers'],
                            'dataset':name})
    if path == 'LEval':
        dataset = load_dataset('L4NLP/LEval', name=name  ,split='test', cache_dir=cache_dir, streaming= True)
        for obj in dataset:
            new_dataset.append({"input": obj['instructions'], "length": len(tokenizer.encode(obj['input'])), 
                            'context':obj['input'] , 'answers':obj['outputs'],
                            'dataset':name})
    return new_dataset

In [92]:
datasets=list()
# 'gov_report' 'qmsum'      /'MultiNews'
for name in ("qasper", "multifieldqa_en", "narrativeqa"):
    dataset = get_dataset('LongBench',name)
    datasets.extend(dataset)
for name in ('multidoc_qa', 'legal_contract_qa', 'financial_qa', 'natural_question', 'scientific_qa' ):
    dataset = get_dataset('LEval',name)
    datasets.extend(dataset)
datasets_id = [(data['length'], i) for i, data in enumerate(datasets)]

In [93]:
len(datasets_id)

648

In [94]:
length =8*(2**10)
rows = 4
new_datasets, choices = [], []
import random
for i in range(rows):
    l = length
    while l > 0:
        datasets_id_tmp = [data_id for data_id in datasets_id if data_id[0]<l]
        if len(datasets_id_tmp)<=0: break
        choice = random.choice(datasets_id_tmp)
        choices.append(choice)
        l = l-choice[0]
    new_datasets.append(choices)
    choices = []

In [95]:
for choices in new_datasets:
    print(len(choices))
    print(sum([c[0] for c in choices]), length)
    context = ''
    for i, c in enumerate(choices):
       context = context + f'Passage{i+1}:\n' + datasets[c[1]]['context'] + '\n\n'
    print(context)
    break

2
7779 8192
Passage1:
Introduction
In this publication, we present Sentence-BERT (SBERT), a modification of the BERT network using siamese and triplet networks that is able to derive semantically meaningful sentence embeddings. This enables BERT to be used for certain new tasks, which up-to-now were not applicable for BERT. These tasks include large-scale semantic similarity comparison, clustering, and information retrieval via semantic search.
BERT set new state-of-the-art performance on various sentence classification and sentence-pair regression tasks. BERT uses a cross-encoder: Two sentences are passed to the transformer network and the target value is predicted. However, this setup is unsuitable for various pair regression tasks due to too many possible combinations. Finding in a collection of $n=10\,000$ sentences the pair with the highest similarity requires with BERT $n\cdot (n-1)/2=49\,995\,000$ inference computations. On a modern V100 GPU, this requires about 65 hours. Simila